In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader, PyMuPDFLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [12]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [21]:
loader = TextLoader("history.txt")
docs = loader.load()

In [38]:
file_path = 'korean_civil_law.pdf'
loader = PyMuPDFLoader(file_path)
docs = loader.load()

In [39]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, length_function=len)
splits = text_splitter.split_documents(docs)

In [40]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [41]:
retriever = vectorstore.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 5, 'lambda_mult': 0.15}
)
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [42]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [57]:
q = "등기사항은 뭐 있어? 관련된 법 조항 번호도 꼭 알려줘"

In [58]:
rag_chain.invoke(q)

'등기사항은 목적, 명칭, 사무소, 설립허가의 연월일, 존립시기나 해산이유를 포함합니다. 등기사항에 대한 규정은 경매의 경우에는 적용되지 않습니다. 해당 법 조항 번호를 알 수 없습니다.'

In [59]:
llm.invoke(q)

AIMessage(content='등기사항에는 다음과 같은 내용이 포함됩니다:\n\n1. 소유자의 성명 또는 상호\n2. 소유자의 주소\n3. 부동산의 지번 및 면적\n4. 등기번호\n5. 부동산의 이용 목적\n6. 권리의 종류 (소유권, 가압류 등)\n7. 권리의 설정일자\n8. 권리의 내용\n9. 권리의 범위\n\n등기사항은 부동산 등기법 제12조에 따라 등기부에 등록되며, 관련 법 조항은 대한민국 부동산 등기법 제12조입니다.', response_metadata={'token_usage': {'completion_tokens': 195, 'prompt_tokens': 40, 'total_tokens': 235}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-daab836b-1f90-43b5-bbc3-d8db3e054117-0')